# 🚜 Predicting the Sale Price of Bulldozers using Machine Learning 

In this notebook, we're going to go through an example machine learning project with the goal of predicting the sale price of bulldozers.

Since we're trying to predict a number, this kind of problem is known as a **regression problem**.

The data and evaluation metric we'll be using (root mean square log error or RMSLE) is from the [Kaggle Bluebook for Bulldozers competition](https://www.kaggle.com/c/bluebook-for-bulldozers/overview).


## 1. Problem Definition

For this dataset, the problem we're trying to solve, or better, the question we're trying to answer is,

> How well can we predict the future sale price of a bulldozer, given its characteristics previous examples of how much similar bulldozers have been sold for?

## 2. Data

Looking at the [dataset from Kaggle](https://www.kaggle.com/c/bluebook-for-bulldozers/data), you can you it's a time series problem. This means there's a time attribute to dataset.

In this case, it's historical sales data of bulldozers. Including things like, model type, size, sale date and more.

**Bulldozer.csv** - Historical bulldozer sales examples up to 2012 (close to 400,000 examples with 50+ different attributes, including `SalePrice` which is the **target variable**).

## 3. Evaluation

For this problem, [Kaggle has set the evaluation metric to being root mean squared log error (RMSLE)](https://www.kaggle.com/c/bluebook-for-bulldozers/overview/evaluation). As with many regression evaluations, the goal will be to get this value as low as possible.

To see how well our model is doing, we'll calculate the RMSLE and then compare our results to others on the [Kaggle leaderboard](https://www.kaggle.com/c/bluebook-for-bulldozers/leaderboard).

## 4. Features

Features are different parts of the data. During this step, you'll want to start finding out what you can about the data.

One of the most common ways to do this, is to create a **data dictionary**.

For this dataset, Kaggle provide a data dictionary which contains information about what each attribute of the dataset means. You can [download this file directly from the Kaggle competition page](https://www.kaggle.com/c/bluebook-for-bulldozers/download/Bnl6RAHA0enbg0UfAvGA%2Fversions%2FwBG4f35Q8mAbfkzwCeZn%2Ffiles%2FData%20Dictionary.xlsx) (account required) or view it on Google Sheets.

With all of this being known, let's get started! 


### Importing the data and preparing it for modelling

In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("./data/Bulldozer.csv", low_memory=False, index_col= 0)
data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
SalesID,,,,,,,,,,,,,,,,,,,,,
1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 1139246 to 6333349
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalePrice                 412698 non-null  float64
 1   MachineID                 412698 non-null  int64  
 2   ModelID                   412698 non-null  int64  
 3   datasource                412698 non-null  int64  
 4   auctioneerID              392562 non-null  float64
 5   YearMade                  412698 non-null  int64  
 6   MachineHoursCurrentMeter  147504 non-null  float64
 7   UsageBand                 73670 non-null   object 
 8   saledate                  412698 non-null  object 
 9   fiModelDesc               412698 non-null  object 
 10  fiBaseModel               412698 non-null  object 
 11  fiSecondaryDesc           271971 non-null  object 
 12  fiModelSeries             58667 non-null   object 
 13  fiModelDescriptor         74816 non-n

In [6]:
data = pd.read_csv("./data/Bulldozer.csv", low_memory=False, index_col= 0, parse_dates=["saledate"])
data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
SalesID,,,,,,,,,,,,,,,,,,,,,
1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,521D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,950FII,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,S175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 1139246 to 6333349
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalePrice                 412698 non-null  float64       
 1   MachineID                 412698 non-null  int64         
 2   ModelID                   412698 non-null  int64         
 3   datasource                412698 non-null  int64         
 4   auctioneerID              392562 non-null  float64       
 5   YearMade                  412698 non-null  int64         
 6   MachineHoursCurrentMeter  147504 non-null  float64       
 7   UsageBand                 73670 non-null   object        
 8   saledate                  412698 non-null  datetime64[ns]
 9   fiModelDesc               412698 non-null  object        
 10  fiBaseModel               412698 non-null  object        
 11  fiSecondaryDesc           271971 non-null  object        


In [8]:
data.sort_values(by="saledate", inplace=True)
data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,TD20,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,A66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,D7G,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,A62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,D3B,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Lever,NaN,NaN


In [9]:
data["sale_year"] = data.saledate.dt.year
data["sale_month"] = data.saledate.dt.month
data["sale_day"] = data.saledate.dt.day

data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,TD20,...,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,17
1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,A66,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,1989,1,31
1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,D7G,...,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,31
1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,A62,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,1989,1,31
1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,D3B,...,NaN,NaN,None or Unspecified,PAT,Lever,NaN,NaN,1989,1,31


In [10]:
data["sale_day_of_year"] = data.saledate.dt.day_of_year
data["sale_day_of_week"] = data.saledate.dt.day_of_week
data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day,sale_day_of_year,sale_day_of_week
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,TD20,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,17,17,1
1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,A66,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,31,1
1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,D7G,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,31,31,1
1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,A62,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,31,1
1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,D3B,...,None or Unspecified,PAT,Lever,NaN,NaN,1989,1,31,31,1


In [12]:
del data["saledate"]

In [13]:
data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day,sale_day_of_year,sale_day_of_week
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,TD20,TD20,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,17,17,1
1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,A66,A66,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,31,1
1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,D7G,D7,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,31,31,1
1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,A62,A62,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,31,1
1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,D3B,D3,...,None or Unspecified,PAT,Lever,NaN,NaN,1989,1,31,31,1


In [14]:
pd.api.types.is_string_dtype(data.UsageBand)

True

In [15]:
for col_name, content in data.iteritems():
    if pd.api.types.is_string_dtype(content):
        data[col_name] = content.astype("category")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 1646770 to 6283635
Data columns (total 56 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   SalePrice                 412698 non-null  float64 
 1   MachineID                 412698 non-null  int64   
 2   ModelID                   412698 non-null  int64   
 3   datasource                412698 non-null  int64   
 4   auctioneerID              392562 non-null  float64 
 5   YearMade                  412698 non-null  int64   
 6   MachineHoursCurrentMeter  147504 non-null  float64 
 7   UsageBand                 73670 non-null   category
 8   fiModelDesc               412698 non-null  category
 9   fiBaseModel               412698 non-null  category
 10  fiSecondaryDesc           271971 non-null  category
 11  fiModelSeries             58667 non-null   category
 12  fiModelDescriptor         74816 non-null   category
 13  ProductSize           

In [17]:
data.Backhoe_Mounting.cat.categories

Index(['None or Unspecified', 'Yes'], dtype='object')

In [18]:
data.Blade_Type.cat.categories

Index(['Angle', 'Coal', 'Landfill', 'No', 'None or Unspecified', 'PAT',
       'Semi U', 'Straight', 'U', 'VPAT'],
      dtype='object')

In [19]:
data.Blade_Type.cat.codes

SalesID
1646770    7
1821514   -1
1505138    7
1671174   -1
1329056    5
          ..
6302984    5
6324811   -1
6313029   -1
6266251   -1
6283635   -1
Length: 412698, dtype: int8

In [21]:
for col_name, content in data.iteritems():
    if pd.api.types.is_categorical_dtype(content):
        data[col_name] = content.cat.codes

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 1646770 to 6283635
Data columns (total 56 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalePrice                 412698 non-null  float64
 1   MachineID                 412698 non-null  int64  
 2   ModelID                   412698 non-null  int64  
 3   datasource                412698 non-null  int64  
 4   auctioneerID              392562 non-null  float64
 5   YearMade                  412698 non-null  int64  
 6   MachineHoursCurrentMeter  147504 non-null  float64
 7   UsageBand                 412698 non-null  int8   
 8   fiModelDesc               412698 non-null  int16  
 9   fiBaseModel               412698 non-null  int16  
 10  fiSecondaryDesc           412698 non-null  int16  
 11  fiModelSeries             412698 non-null  int8   
 12  fiModelDescriptor         412698 non-null  int16  
 13  ProductSize               412698 non-

In [23]:
data["auctioneerID_imputed"] = data.auctioneerID.isna()
data["MachineHoursCurrentMeter_imputed"] = data.MachineHoursCurrentMeter.isna()

data.head()

,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day,sale_day_of_year,sale_day_of_week,auctioneerID_imputed,MachineHoursCurrentMeter_imputed
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,NaN,-1,4592,1743,...,5,-1,-1,1989,1,17,17,1,False,True
1821514,14000.0,1194089,10150,132,99.0,1980,NaN,-1,1819,558,...,-1,3,1,1989,1,31,31,1,False,True
1505138,50000.0,1473654,4139,132,99.0,1978,NaN,-1,2347,712,...,5,-1,-1,1989,1,31,31,1,False,True
1671174,16000.0,1327630,8591,132,99.0,1980,NaN,-1,1818,557,...,-1,3,1,1989,1,31,31,1,False,True
1329056,22000.0,1336053,4089,132,99.0,1984,NaN,-1,2118,682,...,4,-1,-1,1989,1,31,31,1,False,True


In [24]:
data.MachineHoursCurrentMeter.iloc[data.MachineHoursCurrentMeter.isna()] = data.MachineHoursCurrentMeter.mean()
data.head()

C:\Users\ergun\AppData\Local\Temp\ipykernel_24236\439443355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.MachineHoursCurrentMeter.iloc[data.MachineHoursCurrentMeter.isna()] = data.MachineHoursCurrentMeter.mean()


,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day,sale_day_of_year,sale_day_of_week,auctioneerID_imputed,MachineHoursCurrentMeter_imputed
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,9500.0,1126363,8434,132,18.0,1974,3522.988285,-1,4592,1743,...,5,-1,-1,1989,1,17,17,1,False,True
1821514,14000.0,1194089,10150,132,99.0,1980,3522.988285,-1,1819,558,...,-1,3,1,1989,1,31,31,1,False,True
1505138,50000.0,1473654,4139,132,99.0,1978,3522.988285,-1,2347,712,...,5,-1,-1,1989,1,31,31,1,False,True
1671174,16000.0,1327630,8591,132,99.0,1980,3522.988285,-1,1818,557,...,-1,3,1,1989,1,31,31,1,False,True
1329056,22000.0,1336053,4089,132,99.0,1984,3522.988285,-1,2118,682,...,4,-1,-1,1989,1,31,31,1,False,True


In [25]:
data.auctioneerID.value_counts()

1.0     192773
2.0      57441
3.0      30288
4.0      20877
99.0     12042
6.0      11950
7.0       7847
8.0       7419
5.0       7002
10.0      5876
9.0       4764
11.0      3823
12.0      3610
13.0      3068
18.0      2359
14.0      2277
20.0      2238
19.0      2074
16.0      1807
15.0      1742
21.0      1601
22.0      1429
24.0      1357
23.0      1322
17.0      1275
27.0      1150
25.0       959
28.0       860
26.0       796
0.0        536
Name: auctioneerID, dtype: int64

In [26]:
data.auctioneerID.iloc[data.auctioneerID.isna()] = 1.0

C:\Users\ergun\AppData\Local\Temp\ipykernel_24236\3943589175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.auctioneerID.iloc[data.auctioneerID.isna()] = 1.0


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 1646770 to 6283635
Data columns (total 58 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SalePrice                         412698 non-null  float64
 1   MachineID                         412698 non-null  int64  
 2   ModelID                           412698 non-null  int64  
 3   datasource                        412698 non-null  int64  
 4   auctioneerID                      412698 non-null  float64
 5   YearMade                          412698 non-null  int64  
 6   MachineHoursCurrentMeter          412698 non-null  float64
 7   UsageBand                         412698 non-null  int8   
 8   fiModelDesc                       412698 non-null  int16  
 9   fiBaseModel                       412698 non-null  int16  
 10  fiSecondaryDesc                   412698 non-null  int16  
 11  fiModelSeries                     412698 non-

In [28]:
pd.__version__

'1.4.4'

## Model Building

In [31]:
X_train = data.loc[data.sale_year < 2012, "MachineID":] 
X_test = data.loc[data.sale_year == 2012, "MachineID":] 

y_train = data.loc[data.sale_year < 2012, "SalePrice"] 
y_test = data.loc[data.sale_year == 2012, "SalePrice"] 

X_train.head()

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,...,Travel_Controls,Differential_Type,Steering_Controls,sale_year,sale_month,sale_day,sale_day_of_year,sale_day_of_week,auctioneerID_imputed,MachineHoursCurrentMeter_imputed
SalesID,,,,,,,,,,,,,,,,,,,,,
1646770,1126363,8434,132,18.0,1974,3522.988285,-1,4592,1743,-1,...,5,-1,-1,1989,1,17,17,1,False,True
1821514,1194089,10150,132,99.0,1980,3522.988285,-1,1819,558,-1,...,-1,3,1,1989,1,31,31,1,False,True
1505138,1473654,4139,132,99.0,1978,3522.988285,-1,2347,712,58,...,5,-1,-1,1989,1,31,31,1,False,True
1671174,1327630,8591,132,99.0,1980,3522.988285,-1,1818,557,-1,...,-1,3,1,1989,1,31,31,1,False,True
1329056,1336053,4089,132,99.0,1984,3522.988285,-1,2118,682,20,...,4,-1,-1,1989,1,31,31,1,False,True


In [32]:
from sklearn.ensemble import RandomForestRegressor

learner = RandomForestRegressor(n_jobs=-1)


In [33]:
%%time
learner.fit(X_train, y_train)

CPU times: total: 20min 3s
Wall time: 1min 29s


RandomForestRegressor(n_jobs=-1)

In [34]:
import sklearn
sklearn.__version__

'1.2.2'

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true,y_pred))

In [36]:
pred = learner.predict(X_test)
rmsle(y_test, pred)

0.2546388238905036

In [37]:
def CalculatePerformance(model, X_train, X_test, y_train, y_test):
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    results = {"Train R2": r2_score(y_train, pred_train),
               "Test R2": r2_score(y_test, pred_test),
              "Train MAE": mean_absolute_error(y_train, pred_train),
               "Test MAE": mean_absolute_error(y_test, pred_test),
              "Train RMSLE": rmsle(y_train, pred_train),
               "Test RMSLE": rmsle(y_test, pred_test)}
    return results

In [38]:
CalculatePerformance(learner, X_train, X_test, y_train, y_test)

{'Train R2': 0.9875741470733904,
 'Test R2': 0.8719247745806851,
 'Train MAE': 1573.7602620878781,
 'Test MAE': 6114.057707595264,
 'Train RMSLE': 0.08436450443453286,
 'Test RMSLE': 0.2546388238905036}

In [39]:
learner = RandomForestRegressor(n_jobs= -1, max_samples=30000, random_state= 42)

In [40]:
%%time
learner.fit(X_train,y_train)

CPU times: total: 2min 1s
Wall time: 9.49 s


RandomForestRegressor(max_samples=30000, n_jobs=-1, random_state=42)

In [41]:
CalculatePerformance(learner, X_train, X_test, y_train, y_test)

{'Train R2': 0.8978074817788079,
 'Test R2': 0.8594087748719101,
 'Train MAE': 4713.605567092552,
 'Test MAE': 6496.94389181716,
 'Train RMSLE': 0.224046883608472,
 'Test RMSLE': 0.2673656024613305}

In [49]:
params = {
    "n_estimators": [50 ,100, 150,200,250],
    "max_samples": [10000, 20000],
    "max_depth": [None, 5, 10, 20, 30],
    "min_samples_split": [2, 5,10,15],
    "max_features": ["sqrt", 0.5, 0.75, 1.0]
}

In [53]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer

def neg_rmsle(y_true, y_pred):
    return -1*rmsle(y_true, y_pred)

scorer = make_scorer(neg_rmsle)

search_model = RandomizedSearchCV(RandomForestRegressor(), 
                                 param_distributions=params,
                                 n_iter=5,
                                 scoring=scorer,
                                 cv=5,
                                 n_jobs = -1,
                                 verbose =True)

In [54]:
%%time
search_model.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
CPU times: total: 15.6 s
Wall time: 1min 22s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['sqrt', 0.5, 0.75,
                                                         1.0],
                                        'max_samples': [10000, 20000],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250]},
                   scoring=make_scorer(neg_rmsle), verbose=True)

In [56]:
CalculatePerformance(search_model, X_train, X_test, y_train, y_test)

{'Train R2': 0.8711182347114381,
 'Test R2': 0.8444670363619804,
 'Train MAE': 5313.526476316312,
 'Test MAE': 6813.289335306054,
 'Train RMSLE': 0.2476596653838196,
 'Test RMSLE': 0.27831470760098115}

In [46]:
search_model.best_estimator_

RandomForestRegressor(max_depth=20, max_features=0.75, max_samples=30000,
                      min_samples_split=5, n_estimators=150)

In [55]:
results = pd.DataFrame(search_model.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_max_samples,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,63.558025,1.255370,2.463284,0.390126,200,10,10000,0.75,30,"{'n_estimators': 200, 'min_samples_split': 10,...",-0.388472,-0.317604,-0.265093,-0.309479,-0.309062,-0.317942,0.039795,2
1,24.228224,1.604520,0.762476,0.033528,100,15,10000,1.0,5,"{'n_estimators': 100, 'min_samples_split': 15,...",-0.482474,-0.424250,-0.394840,-0.437785,-0.459167,-0.439703,0.029878,5
2,8.544756,0.390703,1.115842,0.088486,50,15,20000,sqrt,10,"{'n_estimators': 50, 'min_samples_split': 15, ...",-0.448148,-0.404994,-0.367463,-0.398159,-0.418188,-0.407390,0.026317,4
3,43.662562,0.569283,6.211806,0.372518,200,2,20000,sqrt,20,"{'n_estimators': 200, 'min_samples_split': 2, ...",-0.405261,-0.337807,-0.290895,-0.326593,-0.350929,-0.342297,0.037277,3
4,39.946455,2.221193,0.823410,0.129681,50,15,20000,1.0,None,"{'n_estimators': 50, 'min_samples_split': 15, ...",-0.381865,-0.305575,-0.257186,-0.305187,-0.291160,-0.308195,0.040830,1


In [57]:
# A Good Model
good_model = RandomForestRegressor(n_estimators  =100, min_samples_leaf = 7, min_samples_split = 4, 
                                  max_features= 0.5, n_jobs = -1, max_depth = None, max_samples =None)
good_model.fit(X_train,y_train)

RandomForestRegressor(max_features=0.5, min_samples_leaf=7, min_samples_split=4,
                      n_jobs=-1)

In [58]:
CalculatePerformance(good_model, X_train, X_test, y_train, y_test)

{'Train R2': 0.9392485219124626,
 'Test R2': 0.8794790674820775,
 'Train MAE': 3554.0469996664106,
 'Test MAE': 5950.141483280706,
 'Train RMSLE': 0.17276096157037943,
 'Test RMSLE': 0.2438809815242914}